In [1]:
# !pip install --upgrade pip
# !pip install transformers datasets evaluate

In [ ]:
import os
import torch
import numpy as np
import evaluate
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset

In [ ]:
GLUE_TESTS = ["mnli", "qqp", "qnli","sst2", "cola", "stsb", "mrpc", "rte", "wnli"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
task = "rte"
dataset = load_dataset("glue", task) # mrpc rte stsb wnli cola ax
print(dataset)
print("train labels :", dataset['train']['label'])
print("val labels :", dataset['validation']['label'])
print("test labels :", dataset['test']['label'])

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/584k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/69.0k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/621k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})
train labels : [1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0

In [ ]:
print(max(dataset['test']['idx']))
print(min(dataset['test']['idx']))

2999
0


In [ ]:
def tokenize_function(dataset_dict):
    return tokenizer(
        dataset_dict["sentence1"],
        dataset_dict["sentence2"],
        padding="max_length",
        truncation=True,
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

Map:   0%|          | 0/277 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5, # default 5e-5
    num_train_epochs=1,
    weight_decay=3e-5,
    logging_dir='./logs',
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.688200,0.659354


TrainOutput(global_step=78, training_loss=0.6911390194526086, metrics={'train_runtime': 231.7112, 'train_samples_per_second': 10.746, 'train_steps_per_second': 0.337, 'total_flos': 655146527846400.0, 'train_loss': 0.6911390194526086, 'epoch': 1.0})

In [ ]:
test_dataset_no_labels = tokenized_datasets['test'].remove_columns('label')

predictions = trainer.predict(test_dataset_no_labels)
probs, labels, _ = predictions
preds = np.argmax(probs, axis=1)
preds
np.any(preds == 1)


True

In [ ]:
print(preds)
print(labels)

[0 0 0 ... 0 0 1]
None


In [ ]:
#metric = evaluate.load("glue", task)
#metric.compute(predictions=preds, references=labels)

In [ ]:
filenames = {
    "cola": "CoLA.tsv",
    "sst2": "SST-2.tsv",
    "mrpc": "MRPC.tsv",
    "qqp": "QQP.tsv",
    "stsb": "STS-B.tsv",
    "mnli_matched": "MNLI-m.tsv",
    "mnli_mismatched": "MNLI-mm.tsv",
    "qnli": "QNLI.tsv",
    "rte": "RTE.tsv",
    "wnli": "WNLI.tsv",
    "ax": "AX.tsv",
}

labelnames = {
    "mnli_matched": ["entailment", "neutral", "contradiction"],
    "mnli_mismatched": ["entailment", "neutral", "contradiction"],
    "ax": ["entailment", "neutral", "contradiction"],
    "qnli": ["entailment", "not_entailment"],
    "rte": ["entailment", "not_entailment"],
}


In [ ]:
ordered_preds = [None] * (max(dataset['test']['idx']) + 1)
for i, pred in zip(dataset['test']['idx'], preds):
    ordered_preds[i] = pred
preds = np.array(ordered_preds)

In [ ]:
submission_directory = "glue_submissions"
if not os.path.exists(submission_directory):
    os.makedirs(submission_directory)
filename = submission_directory + "/" + filenames[task]
labelname = labelnames.get(task)
print(labelname)
print("filename :", filename)

['entailment', 'not_entailment']
filename : glue_submissions/RTE.tsv


In [ ]:
with open(filename, "w") as f:
    f.write("index\tprediction\n")
    for idx, pred in enumerate(preds):
        if pred == None:
            continue
        if labelname:
            pred = labelname[int(pred)]
        f.write(f"{idx}\t{pred}\n")

In [ ]:
!cat glue_submissions/RTE.tsv | head -n 20

index	prediction
0	entailment
1	entailment
2	entailment
3	entailment
4	not_entailment
5	entailment
6	entailment
7	not_entailment
8	entailment
9	entailment
10	entailment
11	entailment
12	entailment
13	entailment
14	entailment
15	entailment
16	not_entailment
17	entailment
18	entailment
